# データ解析特論 2022年度水曜２限

# 第14回 その2 Torch で MLP

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.metrics import classification_report, confusion_matrix

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split, DataLoader

## MNIST

`sklearn.datasets` の MNIST を使う．
解像度の小さいバージョンで学習がしやすいお手軽版．

In [2]:
# MNIST データの読み込み
digits = datasets.load_digits()

# 画像データを配列に変換
X = digits.data
print(X.shape)

# 画像データのピクセル値の範囲を調べる


# X の値域を [0,1] にしておく
X /= np.max(X)

# 画像データに対するラベル
Y = digits.target
print(Y.shape)

(1797, 64)
(1797,)


## PyTorch

### device の設定

In [3]:
print('PyTorch version %s'%torch.__version__ )

print('%s'%torch.backends.cuda.is_built())
print('%s'%torch.cuda.is_available())

# print('%s'%torch.backends.mps.is_built())
# print('%s'%torch.backends.mps.is_available())

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

print(device)

PyTorch version 1.13.1+cu116
True
True
cuda


### パラメータ設定

In [4]:
# 画像サイズ
IMAGE_SIZE = (8, 8)

# Loarder のバッチサイズ
BATCH_SIZE = 32

# 何コア使ってデータを読むか
NUM_WORKERS = 2

# 学習の反復回数
NUM_EPOCHS = 300


## データ準備

### データを PyTorch 用に変換

In [5]:
# Torch のの Tensor 型に変換
Xtensor = torch.tensor(X, dtype=torch.float32)
Ytensor = torch.tensor(Y, dtype=torch.int64)

# label を 1-hot vector 
Ytensor = F.one_hot(Ytensor, num_classes=len(digits.target_names)).float()
print(Ytensor.shape)

# Torch の Dataset
Dataset = TensorDataset(Xtensor, Ytensor)

# Dataset を train-test split
train_size = len(Dataset)*2//3
test_size = len(Dataset) - train_size
train_Dataset, test_Dataset = random_split(Dataset, [train_size, test_size])
print(len(train_Dataset), len(test_Dataset))

torch.Size([1797, 10])
1198 599


### DataLoader の設定

In [6]:
# DataLoader (iterable) を構成
train_loader = DataLoader(dataset=train_Dataset, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=NUM_WORKERS)
test_loader = DataLoader(dataset=test_Dataset, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=NUM_WORKERS)

## モデル構成

ここでは隠れ層を2層持つ MLP を構成しよう．

In [7]:
# 多層パーセプトロンのノード数
d_in = 64
d_h1 = 32
d_h2 = 16
d_out = 10 # Y を 1-hot vector にしておけばしておけば d_out = 10

# モデルのクラス
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(d_in, d_h1)
        self.fc2 = nn.Linear(d_h1, d_h2)
        self.out = nn.Linear(d_h2, d_out)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.out(x)

        return x

# # モデルのインスタンス化
# model = Model().to(device)
# print(model)


## 学習用関数

Torch では 1 epoch 学習する関数を設定して反復させる形態をとる．

In [8]:
def train(model, train_Loader, criterion, optimizer, device='cpu'):
    # loss と 1 epoch で用いたデータのサンプルサイズ用の変数を初期化
    train_loss = 0.0
    n_train = 0
    # model 学習モードに設定
    model.train()
    
    # train_Loader からデータを取り出して loss と勾配を計算
    for Xs, Ys in train_Loader:
        # ミニバッチのサンプルサイズ
        n_train += len(Ys)
        
        # データを device に転送
        Xs, Ys = Xs.to(device), Ys.to(device)
        
        # ミニバッチごとに勾配はリセット
        optimizer.zero_grad()
        
        # model での出力を計算
        outputs = model(Xs)
        
        # loss を計算
        loss = criterion(outputs, Ys)
        
        # 誤差逆伝播
        loss.backward()
        
        # ネットワークのパラメータを更新
        optimizer.step()
        
        # loss の総和の計算のために足しておく
        train_loss += loss.item()
        
    # 1 epoch に用いたサンプルサイズで損失を平均
    train_loss = train_loss / n_train
    
    # return
    return train_loss

def test(model, test_Loader, criterion, optimizer, device='cpu'):
    
    # loss と 1 epoch で用いたデータのサンプルサイズ用の変数を初期化
    test_loss = 0.0
    n_test = 0

    # model を学習できない状態にする
    model.eval()

    # test_Loader からデータを取り出して loss を計算．勾配は計算しない
    with torch.no_grad():
        for Xs, Ys in test_Loader:
            # ミニバッチのサンプルサイズ
            n_test += len(Ys)
            
            # データを device に転送
            Xs, Ys = Xs.to(device), Ys.to(device)
            
            # model での出力を計算 
            outputs = model(Xs)
            
            # loss を計算
            loss = criterion(outputs, Ys)
            
            # loss の総和の計算のために足しておく
            test_loss += loss.item()
        
        # 1 epoch に用いたサンプルサイズで損失を平均   
        test_loss = test_loss / n_test

    return test_loss

## 学習方法の設定

In [9]:
# モデルのインスタンス化
model = Model().to(device)
print(model)

# 損失関数の設定
criterion = nn.CrossEntropyLoss()

# 最適化手法を設定
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

Model(
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (out): Linear(in_features=16, out_features=10, bias=True)
)


## では学習！

In [12]:
%%time
# 1 epoch ごとの loss を格納するリスト
train_loss_list = []
test_loss_list = []

# 学習：NUM_EPOCH 反復
for epoch in range(NUM_EPOCHS):
    
    # 学習
    trainloss = train(model, train_loader, criterion, optimizer, device=device)
    train_loss_list.append(trainloss)
    
    # test で validation
    testloss = test (model, test_loader,  criterion, optimizer, device=device)
    test_loss_list.append(testloss)
    
    # 途中経過の把握のために loss を表示
    if (epoch+1)%10 == 0:
        print('[Epoch %d] train loss = %.5f | test loss = %.5f'%(epoch+1, trainloss, testloss))

[Epoch 10] train loss = 0.00180 | test loss = 0.00410
[Epoch 20] train loss = 0.00170 | test loss = 0.00404
[Epoch 30] train loss = 0.00160 | test loss = 0.00423
[Epoch 40] train loss = 0.00153 | test loss = 0.00393
[Epoch 50] train loss = 0.00145 | test loss = 0.00394
[Epoch 60] train loss = 0.00136 | test loss = 0.00403
[Epoch 70] train loss = 0.00127 | test loss = 0.00389
[Epoch 80] train loss = 0.00121 | test loss = 0.00395
[Epoch 90] train loss = 0.00120 | test loss = 0.00394
[Epoch 100] train loss = 0.00109 | test loss = 0.00397
[Epoch 110] train loss = 0.00104 | test loss = 0.00389
[Epoch 120] train loss = 0.00098 | test loss = 0.00392
[Epoch 130] train loss = 0.00094 | test loss = 0.00395
[Epoch 140] train loss = 0.00092 | test loss = 0.00385
[Epoch 150] train loss = 0.00087 | test loss = 0.00385
[Epoch 160] train loss = 0.00081 | test loss = 0.00393
[Epoch 170] train loss = 0.00079 | test loss = 0.00381
[Epoch 180] train loss = 0.00076 | test loss = 0.00390
[Epoch 190] train l

## 学習結果の確認

### 損失の推移

In [ ]:
# loss の推移の可視化



### 分類の例の図示

In [ ]:
print('training data')


print('test data')



### 定量評価

In [ ]:
# 全データの分類予測を計算


# データを device に送る



### confusion matrix

### classification report

## 課題14-2

`Model` の `forward` を変えて accuracy を上げてみることを試してみよ．

## 以上